# Day 24

In [1]:
with open('../inputs/adventofcode.com_2022_day_24_input.txt', 'r') as f:
    data = f.read().splitlines()

print(f'The board is {len(data)-2} (rows) x {len(data[0])} (cols).')

The board is 25 (rows) x 122 (cols).


In [2]:
# Imports
import numpy as np
from collections import deque

## Puzzle 1

In [3]:
# Create 4 boards, one for each direction of the blizzard
blizzard_l = np.zeros((len(data)-2, len(data[0])-2), dtype=int)
blizzard_r = np.zeros((len(data)-2, len(data[0])-2), dtype=int)
blizzard_u = np.zeros((len(data)-2, len(data[0])-2), dtype=int)
blizzard_d = np.zeros((len(data)-2, len(data[0])-2), dtype=int)

for i, row in enumerate(data[1:-1]):
    for j, col in enumerate(row[1:-1]):
        # Depending on the direction of the blizzard, add it to one of the 4 boards
        if col == "<":
            blizzard_l[i,j] = 1
        elif col == ">":
            blizzard_r[i,j] = 1
        elif col == "^":
            blizzard_u[i,j] = 1
        elif col == "v":
            blizzard_d[i,j] = 1

# Helper function to simulate the movement of the blizzards
def advance_n_steps(bl_l, bl_r, bl_u, bl_d, steps=1):
    
    # Move horizontal blizzards
    bl_l = np.roll(bl_l, -steps, axis=1)
    bl_r = np.roll(bl_r, steps, axis=1)

    # Move vertical blizzards
    bl_u = np.roll(bl_u, -steps, axis=0)
    bl_d = np.roll(bl_d, steps, axis=0)

    return bl_l, bl_r, bl_u, bl_d

# Generate a big number of boards in advanced, so we can look at them later
boards = {}
for i in range(1, 10000):
    blizzard_l, blizzard_r, blizzard_u, blizzard_d = advance_n_steps(blizzard_l, blizzard_r, blizzard_u, blizzard_d)
    board = blizzard_l + blizzard_r + blizzard_u + blizzard_d
    boards[i] = board

In [4]:
# This class represents a node of the graph
# A node is a position in the board, with a minute that it is reached
class Node():
    def __init__(self, row, col, minute):
        self.row = row
        self.col = col
        self.minute = minute

    def __str__(self):
        return f'({self.row}, {self.col}, {self.minute})'

    def __repr__(self):
        return f'({self.row}, {self.col}, {self.minute})'

    def __eq__(self, other):
        return self.row == other.row and self.col == other.col and self.minute >= other.minute

In [5]:
board = boards[1]

height = len(board)
width = len(board[0])

# This list contains for each of the tiles, a list of minutes when it was visited, initialized to -1 (never visited)
visited = [[[-1] for _ in range(width)] for _ in range(height)]

# Queue for breath-first search
queue = deque()
queue.append(Node(0,0,0))

while True:
    if i % 100000 == 0:
        print(f'Iteration {i}. The queue has {len(queue)} elements.')

    n = queue.popleft()

    if n.minute in visited[n.row][n.col]:
        continue
    if n.row == height-1 and n.col == width-1:
        print(f'The exit is reached after {n.minute + 2} minutes.') # +2 because we start at minute 0, and end one minute after the exit is reached
        break
    
    board = boards[n.minute+2]

    if n.row+1 < height and board[n.row+1, n.col] == 0:
        # We can move down
        new_node = Node(n.row+1, n.col, n.minute+1)
        if n.minute+1 not in visited[n.row+1][n.col]:
            queue.append(new_node)
    if n.row-1 >= 0 and board[n.row-1, n.col] == 0:
        # We can move up
        new_node = Node(n.row-1, n.col, n.minute+1)
        if n.minute+1 not in visited[n.row-1][n.col]:
            queue.append(new_node)
    if n.col+1 < width and board[n.row, n.col+1] == 0:
        # We can move right
        new_node = Node(n.row, n.col+1, n.minute+1)
        if n.minute+1 not in visited[n.row][n.col+1]:
            queue.append(new_node)
    if n.col-1 >= 0 and board[n.row, n.col-1] == 0:
        # We can move left
        new_node = Node(n.row, n.col-1, n.minute+1)
        if n.minute+1 not in visited[n.row][n.col-1]:
            queue.append(new_node)
    
    # Wait in place
    new_node = Node(n.row, n.col, n.minute+1)
    if board[n.row, n.col] == 0: # and new_node not in queue:
        queue.append(new_node)
    visited[n.row][n.col].append(n.minute)



The exit is reached after 295 minutes.


In [6]:
# Helper finction to draw the board given the 4 blizzard boards
def draw_board(bl_l, bl_r, bl_u, bl_d):
    for i in range(len(bl_l)):
        for j in range(len(bl_l[0])):
            if bl_l[i,j] == 1:
                print("<", end="")
            elif bl_r[i,j] == 1:
                print(">", end="")
            elif bl_u[i,j] == 1:
                print("^", end="")
            elif bl_d[i,j] == 1:
                print("v", end="")
            else:
                print(".", end="")
        print("")

## Puzzle 2

In [7]:
board = boards[1]

height = len(board)
width = len(board[0])

# This list contains for each of the tiles, a list of minutes when it was visited, initialized to -1 (never visited)
visited = [[[-1] for _ in range(width)] for _ in range(height)]

# Queue for breath-first search
queue = deque()
queue.append(Node(0,0,0))

# Variable to keep track of when we reacht eh exit or the entrance
# set to 1 when reaching the exit, then to 2 when reaching the entrance
goal = 0

while True:

    n = queue.popleft()

    if (goal == 0 or goal == 2) and n.row == height and n.col == width-1:
        print(f'The exit is reached after {n.minute + 1} minutes.')
        # Reset the queue
        queue = deque()
        n = Node(n.row, n.col, n.minute)
        visited = [[[-1] for _ in range(width)] for _ in range(height)]
        # If it is the second time we reach the exit, end
        if goal == 2:
            break
        goal = 1

    if n.row == -1 and n.col == 0 and goal == 1:
        print(f'The entrance is reached after {n.minute + 1} minutes.')
        # Reset the queue
        queue = deque()
        n = Node(n.row, n.col, n.minute)
        visited = [[[-1] for _ in range(width)] for _ in range(height)]
        goal = 2
    
    if 0 <= n.row < height and 0 <= n.col < width and n.minute in visited[n.row][n.col]:
        continue
    
    board = boards[n.minute+2]

    if (n.col == width-1 and n.row+1 == height) or (n.row+1 < height and board[n.row+1, n.col] == 0):
        # We can move down
        new_node = Node(n.row+1, n.col, n.minute+1)
        if (n.col == width-1 and n.row+1 == height) or (n.minute+1 not in visited[n.row+1][n.col]):
            queue.append(new_node)
    if (n.col == 0 and n.row-1 == -1) or (n.row-1 >= 0 and board[n.row-1, n.col] == 0):
        # We can move up
        new_node = Node(n.row-1, n.col, n.minute+1)
        if (n.col == 0 and n.row-1 == -1) or (n.minute+1 not in visited[n.row-1][n.col]):
            queue.append(new_node)
    if n.col+1 < width and 0 <= n.row < height and board[n.row, n.col+1] == 0:
        # We can move right
        new_node = Node(n.row, n.col+1, n.minute+1)
        if n.minute+1 not in visited[n.row][n.col+1]:
            queue.append(new_node)
    if n.col-1 >= 0 and 0 <= n.row < height and board[n.row, n.col-1] == 0:
        # We can move left
        new_node = Node(n.row, n.col-1, n.minute+1)
        if n.minute+1 not in visited[n.row][n.col-1]:
            queue.append(new_node)
    
    # Wait in place
    new_node = Node(n.row, n.col, n.minute+1)
    if (n.row == -1 and n.col == 0) or (n.row == height and n.col == width-1) or board[n.row, n.col] == 0:
        queue.append(new_node)
    if 0 <= n.row < height and 0 <= n.col < width:
        visited[n.row][n.col].append(n.minute)


The exit is reached after 295 minutes.
The entrance is reached after 556 minutes.
The exit is reached after 851 minutes.
